In [0]:
from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
import time

In [0]:
from tensorflow.python.client import device_lib      # uniquement pour vérifier que l'on est bien sur un environement d'execution GPU
print(device_lib.list_local_devices())

# Fonctions
## Fractionnement de la série en ensembles train et test

In [0]:
# split a univariate dataset into train/test sets
def split_dataset(data):
  # split into weeks
  # split_indice=min( (dataset.index[dataset['dates'].str.contains("2018")].tolist()))    # Il ne resulte pas forcément un nombre entier de blocs de 7 jours avec une telle séparation, d'où les traitements qui suivent (combler le datset et le choix des indices ici-bas)
  split_indice= 735          # les 105 premières semaines (<=> jusqu'au 05/01/2018 ) sont prises comme ensemble train et les 52 restantes (<=>à partir de 06/01/2018 ) comme ensemble test 
  train,test = data[0:split_indice], data[split_indice:data.shape[0]]
  # restructure into windows of 7 days              
  train = array(split(train, len(train)/7))
  test = array(split(test, len(test)/7))
  return train, test

## Conversion de la série en format "séquence-cible" (en un format pour apprentissage supérvisé)

In [0]:
# convert history into inputs and outputs
def to_supervised(train, n_input, n_out=7):
	# flatten data
	data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
	X, y = list(), list()
	in_start = 0
	# step over the entire history one time step at a time
	for _ in range(len(data)):
		# define the end of the input sequence
		in_end = in_start + n_input
		out_end = in_end + n_out
		# ensure we have enough data for this instance
		if out_end <= len(data):
			x_input = data[in_start:in_end, 1]              
			x_input = x_input.reshape((len(x_input), 1))
			X.append(x_input)
			y.append(data[in_end:out_end, 1])
		# move along one time step
		in_start += 1
	return array(X), array(y)


## Construction et entrainement du modèle

In [0]:
# train the model
def build_model(train,config):
  # unpack configuration
  n_input,n_nodes, epochs, batch_size = config
  # prepare data
  train_x, train_y = to_supervised(train, n_input)            
  # print(train_x.shape)   # (722, 7, 1)  
  # print(train_y.shape)     # (722, 7)
  # define parameters 
                                 
  n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
  # define model
  model = Sequential()
  model.add(LSTM(n_nodes, activation='relu', input_shape=(n_timesteps, n_features)))
  model.add(Dense(100, activation='relu'))          
  model.add(Dense(n_outputs))
  model.compile(loss='mse', optimizer='adam')
  # fit network
  start=time.time()
  model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=5)

  print("Temps d'entrainement =   %s" %(time.time()-start))
  time.sleep(1)
  return model
  

## Calcul des erreurs de prédiction

In [0]:
# evaluate one or more weekly forecasts against expected values
def evaluate_forecasts(actual, predicted):
	scores = list()
	# calculate an RMSE score for each day
	for i in range(actual.shape[1]):
		# calculate mse
		mse = mean_squared_error(actual[:, i], predicted[:, i])
		# calculate rmse
		rmse = sqrt(mse)
		# store
		scores.append(rmse)
	# calculate overall RMSE
	s = 0
	for row in range(actual.shape[0]):
		for col in range(actual.shape[1]):
			s += (actual[row, col] - predicted[row, col])**2
	score = sqrt(s / (actual.shape[0] * actual.shape[1]))
	return score, scores

## Affichage des scores

In [0]:
# summarize scores
def summarize_scores(name, score, scores):
	s_scores = ', '.join(['%.1f' % s for s in scores])
	print('Résumé scores : modèle : %s: score global [%.3f], scores partiels  %s' % (name, score, s_scores))

## Prédiction 

In [0]:
# make a forecast
def forecast(model, history, n_input):
	# flatten data
	data = array(history)
	data = data.reshape((data.shape[0]*data.shape[1], data.shape[2]))
	# retrieve last observations for input data
	input_x = data[-n_input:, 1]
	# reshape into [1, n_input, 1]
	input_x = input_x.reshape((1, len(input_x), 1))
	# forecast the next week
	yhat = model.predict(input_x, verbose=0)
	# we only want the vector forecast
	yhat = yhat[0]
	return yhat

## Evaluation du modèle (validation "walk-forward" pour chaque bloc de 7 jours)

In [0]:
# evaluate a single model
def evaluate_model(model,train, test, n_input):
	# history is a list of weekly data
	history = [x for x in train]
	# walk-forward validation over each week
	predictions = list()
	for i in range(len(test)):
		# predict the week
		yhat_sequence = forecast(model, history, n_input)
		# store the predictions
		predictions.append(yhat_sequence)
		# get real observation and add to history for predicting the next week
		history.append(test[i, :])
	# evaluate predictions days for each week
	predictions = array(predictions)
	score, scores = evaluate_forecasts(test[:, :, 1], predictions)
	return score, scores

# Chargement des données

In [0]:
dataset = read_csv('/content/drive/My Drive/TER/pass.csv', sep=';',usecols=["dates","nb"], low_memory=False, infer_datetime_format=True)

In [28]:
print(dataset.shape)
print(dataset)
print(dataset[dataset.isna().any(axis=1)])

(1099, 2)
           dates    nb
0     01/01/2016  4618
1     02/01/2016  4925
2     03/01/2016  4325
3     04/01/2016  4528
4     05/01/2016  4134
...          ...   ...
1094  30/12/2018  4729
1095  31/12/2018  4251
1089  25/12/2018  4247
1090  26/12/2018  4715
1091  27/12/2018  4761

[1099 rows x 2 columns]
Empty DataFrame
Columns: [dates, nb]
Index: []


# Combler le dernier bloc de 4 jours avec les 3 dernières valeurs du bloc de 7 jour précédent

In [0]:
dataset= dataset.append(dataset.iloc[dataset.shape[0]-7:dataset.shape[0]-4,])

In [29]:
print(dataset.shape)
print(dataset.iloc[dataset.shape[0]-14:dataset.shape[0],])
print("nombre de bloc de 7 jours: ", dataset.shape[0]/7)  #157

(1099, 2)
           dates    nb
1085  21/12/2018  4392
1086  22/12/2018  4684
1087  23/12/2018  4752
1088  24/12/2018  4342
1089  25/12/2018  4247
1090  26/12/2018  4715
1091  27/12/2018  4761
1092  28/12/2018  4607
1093  29/12/2018  4886
1094  30/12/2018  4729
1095  31/12/2018  4251
1089  25/12/2018  4247
1090  26/12/2018  4715
1091  27/12/2018  4761
nombre de bloc de 7 jours:  157.0


Remarque: les blocs de 7 jours ne sont pas alignés sur une semaine standard,i.e que jour 1 ne correspond pas forcément à un lundi par exemple. D'où la réference bloc de 7 ours et pas une semaine.

# ensembles d'entrainement et de test

In [30]:
train, test = split_dataset(dataset.values)
# validate train data
print(train.shape)
print("du ",train[0, 0, 0],"au", train[-1, -1, 0])
# validate test
print(test.shape)
print("du ",test[0, 0, 0],"au", test[-1, -4, 0],"+ 4 jours")

(105, 7, 2)
du  01/01/2016 au 04/01/2018
(52, 7, 2)
du  05/01/2018 au 31/12/2018 + 4 jours


# Execution
**Entrées**: fichier csv de la série temporelle.

Un modèle sauvegardé peut etre rechargé. 
Un autre modèle peut etre réajusté:

**Paramètres à renseigner** : (dans la cellule Entrainement et évaluation du modèle) 

- n_input : nombre d'observations antérieures à considérer.
- n_nodes : nombre de blocs LSTM
- n_epochs : nombre d'epochs
- n_batch : taille du batch
  
Bien que la taille de la séquence prédite peut etre paramétrée (pour le modèle et la fonction to_supervised()) , la fonction fractionnant les données (split_dataset()) devra etre modifiée pour d'autres tailles.
La taille de la séquence antérieure peut elle etre paramétrée ( par expl. l'on peut choisir de prédir les 7 jour futurs étant donné 14 observations passées).

Les tailles des ensembles d'entrainement et de test ont été fixées de sorte à considérer pratiquement les deux premières années pour l'entrainement et la dernière pour le test, mais aussi de sorte à avoir des blocs de 7 jours dans chaque ensemble, si d'autres proportions sont à choisir, les modifications conséquentes sont à apporter à la fonction split_dataset() notamment.

**Sorties**: le modèle peut etre sauvegardé.
Ce modèle peut etre invoqué pour calculer les prédictions d'un ensemble de données (ici seule l'évaluation Walk Forward est faite, le RMSE global et les RMSEs partiels, pour chaque jour du bloc de 7 jours sont retournés ) 

Le modèle retourne les prédictions (t (n_input +1)- t (n_input +7)) pour une séquence antérieure (t(1) - t(n_input))

# Entrainement et évaluation du modèle

In [31]:
n_input = 7
n_nodes= 200
epochs= 70
batch_size= 16   
# fit model
config= (n_input,n_nodes,epochs, batch_size )
model = build_model(train, config)
# evaluate model and get scores
score, scores = evaluate_model(model,train, test, n_input)
# summarize scores
summarize_scores('lstm', score, scores)
# plot scores
days = ['jour 1', 'jour 2', 'jour 3', 'jour 4', 'jour 5', 'jour 6', 'jour 7']
pyplot.plot(days, scores, marker='o', label='lstm')
pyplot.show()

Epoch 1/70
Epoch 2/70
Epoch 3/70
Epoch 4/70
Epoch 5/70


KeyboardInterrupt: ignored

# Sauvegarde du modèle

In [0]:
# serialize model to JSON
model_json = model.to_json()
with open('/content/drive/My Drive/lstm_multistep.json', "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights('/content/drive/My Drive/lstm_multistep.h5')


# Chargement du modèle sauvegardé

In [0]:
# load json and create model
json_file = open('/content/drive/My Drive/lstm_multistep.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("/content/drive/My Drive/lstm_multistep.h5")
